## Non Compliance Experiment=1
Test top norms for different w_nc

In [ ]:
%%time
%%capture exp_nc_0_log
!dvc run -n non_compliance_exp \
-d ../scripts/nc_experiments.py -d ../params_nc.yaml -d ../data/env.pickle\
-o ../data_nc/$exp_dir_name/  \
python ../scripts/nc_experiments.py
!dvc add ../data_nc/$exp_dir_name/
!dvc push
!git add --all
!git commit -m "Results gen for exp_nc_0"

In [33]:
%exp_nc_0_log.show()

ERROR: '../data_nc' is busy, it is being blocked by:
  (PID 2765): /Users/ashish1610dhiman/data_projects/otago_norm_learning/mcmc/venv/bin/dvc run -n non_compliance_exp -d ../scripts/nc_experiments.py -d ../params_nc.yaml -d ../data/env.pickle -o ../data_nc// python ../scripts/nc_experiments.py

If there are no processes with such PIDs, you can manually remove'.dvc/tmp/rwlock' and try again.


In [ ]:
%store exp_nc_0_log > ../data_nc/exp0/exp_nc_0.log

In [16]:
from tqdm.notebook import tqdm

In [17]:
w_nc_list=[0.0,0.05,0.1,0.2,0.3,0.4]

In [ ]:
for w_nc in tqdm(w_nc_list,"Loop Over diff w_nc"):
    exp_dir_name="exp_nc_1.w_nc={}".format(w_nc)
    !dvc run -n non_compliance_exp \
        -d ../scripts/nc_experiments.py -d ../params_nc.yaml -d ../data/env.pickle\
        -o ../data_nc/$exp_dir_name  \
        python ../scripts/nc
    !dvc add
    !dvc push
    !git add --all
    !git commit -m result gen for {exp_dir_name}